## Results Section Analysis

This notebook provides a structured workflow to generate the tables, descriptive statistics, plots, and statistical tests needed for the Results section of your ML report. 

**Instructions:**
- Place your metrics file (`metrics.csv`) and prediction file (`predictions.csv`) in the same folder as this notebook.
- The `metrics.csv` file should contain: `fold`, `Model`, `Metric1`, `Metric2`, `Metric3`.
- The `predictions.csv` file should contain: `fold`, `Model`, `y_true`, `y_pred`, `y_score` for ROC curves.


In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix
import scipy.stats as stats
from statsmodels.stats.weightstats import DescrStatsW


### Load Data

In [ ]:
# Load your data
metrics = pd.read_csv('metrics.csv')
predictions = pd.read_csv('predictions.csv')
# Peek at the data
display(metrics.head())
display(predictions.head())


### Descriptive Statistics (Table 1)

In [ ]:
# Compute descriptive statistics for each metric and model
desc = metrics.groupby('Model').agg({
    'Metric1': ['mean', 'std'],
    'Metric2': ['mean', 'std'],
    'Metric3': ['mean', 'std']
})
desc.columns = ['_'.join(col) for col in desc.columns]
desc.reset_index(inplace=True)
desc

### Confusion Matrices (Figure 4)

In [ ]:
# Plot confusion matrices for each model
models = predictions['Model'].unique()
for model in models:
    df = predictions[predictions['Model'] == model]
    cm = confusion_matrix(df['y_true'], df['y_pred'])
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title(f'Confusion Matrix: {model}')
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha='center', va='center')
    plt.savefig(f'confusion_matrix_{model}.png')
    plt.show()


### Boxplots of Metric Distributions (Figure 5)

In [ ]:
# Create boxplots for each metric
for metric in ['Metric1', 'Metric2', 'Metric3']:
    plt.figure()
    metrics.boxplot(column=metric, by='Model', grid=False)
    plt.title(f'Boxplot of {metric} by Model')
    plt.suptitle('')  # remove automatic title
    plt.xlabel('Model')
    plt.ylabel(metric)
    plt.savefig(f'boxplot_{metric}.png')
    plt.show()


### Statistical Tests (p-values & Confidence Intervals)

In [ ]:
# Example: paired t-test and 95% CI for Metric1 between two models
model_pairs = [('Method1', 'Method2')]
results = []
for m1, m2 in model_pairs:
    d1 = metrics[metrics['Model'] == m1]['Metric1']
    d2 = metrics[metrics['Model'] == m2]['Metric1']
    # Paired t-test
    t_stat, p_val = stats.ttest_rel(d1, d2)
    # 95% CI for difference
    diff = d1 - d2
    ci_low, ci_upp = DescrStatsW(diff).tconfint_mean()
    results.append({'Comparison': f'{m1} vs {m2}', 'p-value': p_val, 'CI_low': ci_low, 'CI_up': ci_upp})
results_df = pd.DataFrame(results)
results_df

### ROC Curves

In [ ]:
# Plot ROC curves for each model
plt.figure()
for model in models:
    df = predictions[predictions['Model'] == model]
    fpr, tpr, _ = roc_curve(df['y_true'], df['y_score'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{model} (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.savefig('roc_curves.png')
plt.show()
